# Planning my next vacay ☀️

This project uses APIs and web-scraping techniques to determine the plus beau places to spontaneously visit.

Here are the 35 best places to visit in France in 2020 based on <a href="https://one-week-in.com/35-cities-to-visit-in-france/" target="_blank">this website</a>:

```python
["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]
```

1. Get coordinates of all cities with <a href="https://nominatim.org/" target="_blank">https://nominatim.org/</a>.

    Documentation: <a href="https://nominatim.org/release-docs/develop/api/Search/" target="_blank">https://nominatim.org/release-docs/develop/api/Search/</a>


2. Use <a href="https://openweathermap.org/appid" target="_blank">https://openweathermap.org/appid</a> (you have to subscribe to get a free apikey) and <a href="https://openweathermap.org/api/one-call-api" target="_blank">https://openweathermap.org/api/one-call-api</a> to get weather info about the above cities and put it in a DataFrame.


3. Determing which cities will have the nicest weather over the next week.


4. Save everything results to a csv file.


5. Use plotly to display the best destinations on a map.

In [1]:
import requests
import pandas as pd
import plotly.express as px

In [2]:
city_names = [
    "Mont Saint Michel",
    "St Malo",
    "Bayeux",
    "Le Havre",
    "Rouen",
    "Paris",
    "Amiens",
    "Lille",
    "Strasbourg",
    "Chateau du Haut Koenigsbourg",
    "Colmar",
    "Eguisheim",
    "Besancon",
    "Dijon",
    "Annecy",
    "Grenoble",
    "Lyon",
    "Gorges du Verdon",
    "Bormes les Mimosas",
    "Cassis",
    "Marseille",
    "Aix en Provence",
    "Avignon",
    "Uzes",
    "Nimes",
    "Aigues Mortes",
    "Saintes Maries de la mer",
    "Collioure",
    "Carcassonne",
    "Foix",
    "Toulouse",
    "Montauban",
    "Biarritz",
    "Bayonne",
    "La Rochelle"
]

In [3]:
df = pd.DataFrame(columns=['city_id' , 'name', 'latitude', 'longitude', 'main_weather', 'expected_rain', 'day_temperature'])

In [ ]:
API_KEY = "YOUR_API_KEY"

In [ ]:
for i in range(len(city_names)):
    
    print("Making requests for {}".format(city_names[i]))
    
    # Use nominatim api to get city coordinates
    par = {
        "city": city_names[i],
        "country": "France",
        "format": "json"
    }
    
    r = requests.get('https://nominatim.openstreetmap.org/search', params=par)
    res = r.json()
    
    df.loc[i,'city_id'] = i
    df.loc[i,'name'] = city_names[i]
    df.loc[i, 'latitude'] = res[0]['lat']
    df.loc[i, 'longitude'] = res[0]['lon']
    
    # Use openweathermap api to get weather for next seven days
    par = {
        "lat": df.loc[i, 'latitude'],
        "lon": df.loc[i, 'longitude'],
        "exclude": "current,minutely,hourly",
        "units": "metric",
        "appid": API_KEY,
    }

    r = requests.get('https://api.openweathermap.org/data/2.5/onecall', params=par)
    res = r.json()
    
    # Compute expected volume of rain
    expected_rain = 0
    print(res)
    for d in res['daily']:
        if 'rain' in d.keys():
            expected_rain += d['pop']*d['rain']
            
    # Compute average day temperature
    temperatures = pd.Series([d['temp']['day'] for d in res['daily']])
    mean_temperature = temperatures.mean()

    # Extract most probable weather
    weathers = pd.Series([d['weather'][0]['main'] for d in res['daily']])
    main_weather = weathers.mode()[0]
    
    df.loc[i, 'main_weather'] = main_weather
    df.loc[i,'expected_rain'] = expected_rain
    df.loc[i,'day_temperature'] = mean_temperature
    

In [ ]:
df.loc[:,'rank'] = df['expected_rain'].rank(method='min')
df.loc[:,'inverted_rank'] = df['expected_rain'].rank(method='min', ascending=False)

df = df.sort_values(by=['expected_rain', 'day_temperature'], ascending = [True, False]).reset_index(drop=True)
display(df)

In [ ]:
print('Best cities to visit next week: ')
print()

for i,row in df.loc[df['rank']==1,:].iterrows():
    print("{} -- Mostly {} with temperature {} °C".format(row['name'], row['main_weather'], row['day_temperature']))

In [ ]:
df.loc[:,'latitude'] = df['latitude'].astype('float')
df.loc[:,'longitude'] = df['longitude'].astype('float')
df.loc[:,'expected_rain'] = df['expected_rain'].astype('float')
df.loc[:,'day_temperature'] = df['day_temperature'].astype('float')
df.loc[:,'rank'] = df['rank'].astype('int')
df.loc[:,'inverted_rank'] = df['inverted_rank'].astype('int')

In [ ]:
fig = px.scatter_mapbox(df, lat="latitude", lon="longitude", hover_name = 'name', zoom = 4,
                        hover_data = ['main_weather', 'expected_rain', 'day_temperature'], 
                        color = 'day_temperature', color_continuous_scale = 'Bluered', size = 'inverted_rank',
                        mapbox_style="carto-positron")
fig.show()

In [ ]:
df.to_csv('res/1_destinations.csv', index=False)